<a href="https://colab.research.google.com/github/Batwan01/2024-Challenge/blob/main/history/24-9-9/MLP5%2BPReLU_base_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.3 MB/s eta 0:00:00


In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, ElasticNet, Lars, LassoLars, OrthogonalMatchingPursuit, BayesianRidge, ARDRegression, PassiveAggressiveRegressor, RANSACRegressor, HuberRegressor)
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import BaggingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# 재현 가능성을 위한 시드 고정
RANDOM_SEED = 18
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)


In [5]:
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
# 데이터 로드 및 전처리
train_csv_path = "/content/drive/MyDrive/Colab Notebooks/contest/samsung/train.csv"
test_csv_path = "/content/drive/MyDrive/Colab Notebooks/contest/samsung/test.csv"
train = pd.read_csv(train_csv_path)
test = pd.read_csv(test_csv_path)
com = pd.concat([train, test])
com = com.drop(['x_2', 'x_6'], axis=1)

train_data = com[:40118]
X_test = com[40118:].drop('y', axis=1).iloc[:, 1:]

# 입력 데이터와 라벨 분리
X_train_df = train_data.drop(['ID', 'y'], axis=1)
y_train_df = train_data['y']

# 70 미만 값 제거
mask = y_train_df >= 70
X_train = X_train_df[mask]
y_train = y_train_df[mask]

print(X_test.shape)
print(X_train.shape)

(4986, 9)
(40110, 9)


In [44]:
from xgboost import XGBRegressor
# NRMSE 계산 함수
def lg_nrmse(gt, preds):
    all_nrmse = []
    for idx in range(gt.shape[1]):
        rmse = mean_squared_error(gt[:, idx], preds[:, idx], squared=False)
        nrmse = rmse / np.mean(np.abs(gt[:, idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

# 기본 모델들 정의
base_models = [
    ('HistGradientBoosting', HistGradientBoostingRegressor(random_state=RANDOM_SEED)),
    ('ExtraTrees', ExtraTreesRegressor(n_jobs=-1, random_state=RANDOM_SEED)),
    ('RandomForest', RandomForestRegressor(n_jobs=-1, random_state=RANDOM_SEED)),
    ('LightGBM', LGBMRegressor(n_jobs=-1, random_state=RANDOM_SEED, force_col_wise=True)),
    ('XGBoost', XGBRegressor(tree_method='hist', device='cuda', random_state=RANDOM_SEED, n_jobs=-1))  # XGBoost 추가 및 GPU 사용
]

In [48]:
# 최종 모델 정의 (meta-model)
final_model = LinearRegression()

# Stacking Regressor 정의
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=final_model
)

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# KFold 교차 검증 설정
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Mean Squared Error를 평가 지표로 사용
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# 교차 검증 수행
cv_scores = cross_val_score(stacking_regressor, X_train, y_train, cv=kfold, scoring=scorer)

# 평균 MSE 출력
print(f"Mean Squared Error (negative) for each fold: {cv_scores}")
print(f"Mean Squared Error (negative) across folds: {-cv_scores.mean():.4f}")

[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 32088, number of used features: 9
[LightGBM] [Info] Start training from score 83.650282
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25670, number of used features: 9
[LightGBM] [Info] Start training from score 83.650161
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25670, number of used features: 9
[LightGBM] [Info] Start training from score 83.651586
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25670, number of used features: 9
[LightGBM] [Info] Start training from score 83.643993
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25671, number of used features: 9
[LightGBM] [Info] Start training from score 83.664031
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25671, n

In [56]:
stacking_regressor1 = StackingRegressor(
    estimators=base_models,
    final_estimator=final_model
)
stacking_regressor2 = StackingRegressor(
    estimators=base_models,
    final_estimator=final_model
)
stacking_regressor3 = StackingRegressor(
    estimators=base_models,
    final_estimator=final_model
)
stacking_regressor4 = StackingRegressor(
    estimators=base_models,
    final_estimator=final_model
)
stacking_regressor5 = StackingRegressor(
    estimators=base_models,
    final_estimator=final_model
)
stack_list = [stacking_regressor1, stacking_regressor2, stacking_regressor3, stacking_regressor4, stacking_regressor5]
x_train_fold = []
y_train_fold = []

for train_index, test_index in kfold.split(X_train, y_train):
    X = X_train.iloc[train_index] # Use .iloc to select rows by position.
    y = y_train.iloc[train_index] # Use .iloc to select rows by position.
    x_train_fold.append(X) # Use append instead of appned.
    y_train_fold.append(y)


In [58]:
for i in range(5):
    stack_list[i].fit(x_train_fold[i], y_train_fold[i])

[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 32088, number of used features: 9
[LightGBM] [Info] Start training from score 83.646558
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25670, number of used features: 9
[LightGBM] [Info] Start training from score 83.655487
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25670, number of used features: 9
[LightGBM] [Info] Start training from score 83.644303
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25670, number of used features: 9
[LightGBM] [Info] Start training from score 83.646046
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25671, number of used features: 9
[LightGBM] [Info] Start training from score 83.650180
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 25671, n

IndexError: list index out of range

In [60]:
# 모델 학습 (fit) 수행
stacking_regressor.fit(X_train, y_train)

[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 40110, number of used features: 9
[LightGBM] [Info] Start training from score 83.644580
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 32088, number of used features: 9
[LightGBM] [Info] Start training from score 83.646746
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 32088, number of used features: 9
[LightGBM] [Info] Start training from score 83.642818
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 32088, number of used features: 9
[LightGBM] [Info] Start training from score 83.643259
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 32088, number of used features: 9
[LightGBM] [Info] Start training from score 83.655282
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 32088, n

StackingRegressor(estimators=[('HistGradientBoosting',
                               HistGradientBoostingRegressor(random_state=18)),
                              ('ExtraTrees',
                               ExtraTreesRegressor(n_jobs=-1, random_state=18)),
                              ('RandomForest',
                               RandomForestRegressor(n_jobs=-1,
                                                     random_state=18)),
                              ('LightGBM',
                               LGBMRegressor(force_col_wise=True, n_jobs=-1,
                                             random_state=18)),
                              ('XGBoost',
                               XGBRegressor(base_score=None, booste...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=18, ...))],
                  final_estimator=LinearRegression())

In [64]:
# 훈련 데이터로 성능 평가
y_train_pred = stacking_regressor.predict(X_train)  # 훈련 데이터에 대한 예측
mse_train = mean_squared_error(y_train, y_train_pred)  # 훈련 데이터에서 MSE 계산
print(f"Mean Squared Error on training data: {mse_train:.4f}")

# 테스트 데이터에 대한 예측 (성능 평가 X)
y_pred = stacking_regressor.predict(X_test)  # 테스트 데이터에 대한 예측 수행

# 테스트 데이터의 예측 결과 출력 또는 저장
print(f"Predictions for test data: {y_pred[:5]}")  # 테스트 예측값의 일부를 출력


Mean Squared Error on training data: 2.1603
Predictions for test data: [83.69120843 82.23591382 90.8824177  91.66460792 82.03445883]


In [65]:
# 결과 저장 (1차원 배열 처리)
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/contest/samsung/sample_submission.csv")

# 'y' 열에 예측 결과 할당 (결과가 1차원 배열일 때)
submission['y'] = y_pred  # 'y' 컬럼에 1차원 예측 결과를 넣음

# CSV 파일로 저장
submission.to_csv("/content/drive/MyDrive/Colab Notebooks/contest/samsung/results/Stacking_Predictions.csv", index=False)

print("예측 결과가 CSV 파일로 저장되었습니다.")

예측 결과가 CSV 파일로 저장되었습니다.


In [8]:
def find_non_matching_ids(file1, file2):
    # 두 개의 CSV 파일을 읽어옴
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # y 값 기준으로 내림차순 정렬
    df1_sorted = df1.sort_values(by='y', ascending=False)
    df2_sorted = df2.sort_values(by='y', ascending=False)

    # file1의 상위 10% 항목 계산
    top_10_percent_count = int(len(df1_sorted) * 0.1)
    top_10_percent_ids_df1 = set(df1_sorted.head(top_10_percent_count)['ID'])

    # file2의 상위 10% ID 추출
    top_10_percent_ids_df2 = set(df2_sorted.head(top_10_percent_count)['ID'])

    # file1의 상위 10% 중 file2의 상위 10%에 없는 ID 계산
    non_matching_ids = top_10_percent_ids_df1 - top_10_percent_ids_df2
    num_non_matching = len(non_matching_ids)

    # 결과 출력
    print(f"file1의 상위 10% 항목 개수: {top_10_percent_count}")
    print(f"file1의 상위 10% 중 file2에 없는 항목 개수: {num_non_matching}")
    print(f"file1의 상위 10% 중 file2에 없는 항목 ID: {non_matching_ids}")

    return top_10_percent_count, num_non_matching, list(non_matching_ids)

# 사용 예시
file1 ='/content/drive/MyDrive/Colab Notebooks/contest/samsung/MLP_Residual_Connection_drop_x2_x6(0.752).csv' # best 성능 파일
#file2 = '/content/drive/MyDrive/Colab Notebooks/contest/samsung/MLP_Residual_Connection_18.csv'
file2 = '/content/Stacking_Predictions.csv'
file3 = '/content/drive/MyDrive/Colab Notebooks/contest/samsung/results/Stacking_Predictions.csv' # 측정하고자 하는 파일
#top_10_percent_count, num_non_matching, non_matching_ids = find_non_matching_ids(file1, file3)
top_10_percent_count, num_non_matching, non_matching_ids = find_non_matching_ids(file2, file3)

file1의 상위 10% 항목 개수: 498
file1의 상위 10% 중 file2에 없는 항목 개수: 41
file1의 상위 10% 중 file2에 없는 항목 ID: {'TEST_0917', 'TEST_3403', 'TEST_3889', 'TEST_4221', 'TEST_1592', 'TEST_1015', 'TEST_1690', 'TEST_2733', 'TEST_3838', 'TEST_1162', 'TEST_1803', 'TEST_2937', 'TEST_4069', 'TEST_2064', 'TEST_4738', 'TEST_1377', 'TEST_3734', 'TEST_2765', 'TEST_1978', 'TEST_1839', 'TEST_0823', 'TEST_4543', 'TEST_3718', 'TEST_0713', 'TEST_2418', 'TEST_1904', 'TEST_3209', 'TEST_2865', 'TEST_4682', 'TEST_3625', 'TEST_1327', 'TEST_4472', 'TEST_1335', 'TEST_4828', 'TEST_2351', 'TEST_0864', 'TEST_3329', 'TEST_2125', 'TEST_0548', 'TEST_0197', 'TEST_3065'}


In [67]:
arr = list(non_matching_ids)
# 확인하고자 하는 ID 리스트
ids_to_check = ['TEST_3536', 'TEST_1574', 'TEST_3731', 'TEST_1780', 'TEST_1746', 'TEST_3287', 'TEST_1429', 'TEST_1731']
ids_to_check_false = ['TEST_1377', 'TEST_2035', 'TEST_3042', 'TEST_4949', 'TEST_4578', 'TEST_4707', 'TEST_2538', 'TEST_1909']
# non_matching_ids에 있는지 확인
matching_ids_T = [id_ for id_ in ids_to_check if id_ in arr]
matching_ids_F = [id_ for id_ in ids_to_check_false if id_ in arr]
# 결과 출력
print("정답 예측")
print(f"다음 ID들이 non_matching_ids에 있습니다: {matching_ids_T}")
print(f"non_matching_ids에서 확인된 ID 개수: {len(matching_ids_T)}")

# 결과 출력
print("오답 예측")
print(f"다음 ID들이 non_matching_ids에 있습니다: {matching_ids_F}")
print(f"non_matching_ids에서 확인된 ID 개수: {len(matching_ids_F)}")

정답 예측
다음 ID들이 non_matching_ids에 있습니다: ['TEST_1574']
non_matching_ids에서 확인된 ID 개수: 1
오답 예측
다음 ID들이 non_matching_ids에 있습니다: ['TEST_4949', 'TEST_4578', 'TEST_2538', 'TEST_1909']
non_matching_ids에서 확인된 ID 개수: 4
